# Complex Workflow with AxisVM

In [1]:
from axisvm import examples
import pyvista as pv
import numpy as np

from sigmaepsilon.mesh import PolyData, CartesianFrame, PointData
from sigmaepsilon.mesh.cells import TET4 as CellData
from sigmaepsilon.mesh.tri.triutils import edges_tri
from sigmaepsilon.mesh.topo import unique_topo_data, detach
from sigmaepsilon.mesh.topo import rewire, detach

vtkpath = examples.download_stand_vtk()
pd = PolyData.read(vtkpath)
c = pd.coords() / 1000
t = pd.topology()[:, :4]
c, t = detach(c, t)
csys = CartesianFrame(dim=3)

# pointdata and celldata
pd = PointData(coords=c, frame=csys)
cd = CellData(topo=t, frames=csys)

mesh = PolyData(pd, cd, frame=csys)
surface = mesh.surface()
surface.nummrg()

coords, topo = surface.coords(), surface.topology()
edges, edgeIDs = unique_topo_data(edges_tri(topo))
edges += 1
edgeIDs += 1

PolyData({})

In [5]:
from axisvm.com.client import start_AxisVM
axvm = start_AxisVM(visible=True, daemon=True)

In [6]:
modelId = axvm.Models.New()
axm = axvm.Models.Item[modelId]
axm.Settings.EditingTolerance = -1
wdir = ""

from axisvm.com.tlb import RPoint3d
foo = lambda x : RPoint3d(x=x[0], y=x[1], z=x[2])
axm.BeginUpdate()
axm.Nodes.BulkAdd(list(map(foo, coords)))
axm.EndUpdate()

from axisvm.com.tlb import lgtStraightLine, RLineData

def gen_line(edge):
    return RLineData(
        NodeId1 = edge[0],
        NodeId2 = edge[1],
        GeomType = lgtStraightLine
    )
    
axm.BeginUpdate()
axm.Lines.BulkAdd(list(map(gen_line, edges)))
axm.EndUpdate()

from axisvm.com.tlb import vTop
axm.View = vTop
axm.FitInView()

from axisvm.com.tlb import ndcEuroCode
axm.Settings.NationalDesignCode = ndcEuroCode
matId = axm.Materials.AddFromCatalog(ndcEuroCode, "S 235")

from axisvm.com.tlb import RSurfaceAttr, lnlTensionAndCompression, \
    RResistancesXYZ, schLinear, stShell, RElasticFoundationXYZ, \
    RNonLinearityXYZ, RSurface
    
SurfaceAttr = RSurfaceAttr(
    Thickness=1,
    SurfaceType=stShell,
    RefZId=0,
    RefXId=0,
    MaterialId=matId,
    ElasticFoundation=RElasticFoundationXYZ(0, 0, 0),
    NonLinearity=RNonLinearityXYZ(lnlTensionAndCompression,
                                  lnlTensionAndCompression,
                                  lnlTensionAndCompression),
    Resistance=RResistancesXYZ(0, 0, 0),
    Charactersitics=schLinear)

def gen_surface(edges):
    return RSurface(
        N=3,
        LineIndex1 = edges[0],
        LineIndex2 = edges[1],
        LineIndex3 = edges[2],
        Attr = SurfaceAttr,
        DomainIndex = 0
    )
axm.BeginUpdate()
axm.Surfaces.BulkAdd(list(map(gen_surface, edgeIDs)))
axm.EndUpdate()


0

In [7]:
attachments = axm.Nodes.Attachments
gid = surface.pd.gid
nN = axm.Nodes.Count
for i in range(nN):
    attachments.AddData('gid', i+1, [gid[i]])

Select points of action

In [8]:
from axisvm.com.tlb import RPartItem, pitNode
axvm.BringToFront()
CustomParts = axm.CustomParts
CustomPartsFolder = CustomParts.RootFolder
i = CustomPartsFolder.AddSubFolder('SigmaEpsilon')
Folder = CustomPartsFolder.SubFolder[i]
iebc = Folder.AddPart('EBC', [])[-1]
inbc = Folder.AddPart('NBC', [])[-1]

In [9]:

nodes_f = axm.Nodes.select_IDs(msg='Select nodes where nodal loads are to be imposed!')
parts = list(map(lambda id : RPartItem(ItemType=pitNode, Id=id), nodes_f))
Folder.AddPartItemsToPart(inbc, parts)
axm.SelectAll(False)
axm.Refresh()

0

In [10]:
nodes_u = axm.Nodes.select_IDs(msg='Select nodes where displacement penalties are to be enforced!')
parts = list(map(lambda id : RPartItem(ItemType=pitNode, Id=id), nodes_u))
Folder.AddPartItemsToPart(iebc, parts)
axm.SelectAll(False)
axm.Refresh()

0

In [11]:
nbc_items = Folder.GetPart(inbc)[0]
nbc_ids = list(map(lambda i : i.Id, filter(lambda i : i.ItemType == 0, nbc_items)))
ebc_items = Folder.GetPart(iebc)[0]
ebc_ids = list(map(lambda i : i.Id, filter(lambda i : i.ItemType == 0, ebc_items)))